In [4]:
from snowflake.snowpark.session import Session
from snowflake.snowpark import DataFrame, Window, functions as F
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import col, lit, udf

import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("rsa_key.p8", "rb") as key:
  p_key= serialization.load_pem_private_key(
    key.read(),
    password="snowflake".encode(),
    backend=default_backend()
  )

pkb = p_key.private_bytes(
  encoding=serialization.Encoding.DER,
  format=serialization.PrivateFormat.PKCS8,
  encryption_algorithm=serialization.NoEncryption())

from config import snowflake_conn_prop

snowflake_conn_prop["private_key"] = pkb

session = Session.builder.configs(snowflake_conn_prop).create()

print(session.sql('select current_warehouse(), current_database(), current_schema(), current_user()').collect())